In [1]:
'''
1. Loaded data from IMU Dataset as a pandas dataframe
2. Tried to do in Spark but there is no functionality
in Spark to load .xlsx files. It required a package outside
the Spark ecosystem and I thought it is not worth it.
3. Further there is a richer collection of feature selectors available to us 
in sklearn compared to MLlib. We can switch to Spark once the EDA is done.
'''

import os
from glob import glob
import pandas as pd

def create_df(path):
    count = 1
    for files in path:
       if count == 1:
          df1 = pd.read_excel(files)
       else:    
          df2 = pd.read_excel(files)
          df1 = df1.append(df2, ignore_index=True)
       count = count + 1    
    return df1

falls = glob("IMU Dataset_sample/sub*/Falls/*xlsx")
near_falls = glob("IMU Dataset_sample/sub*/Near_Falls/*xlsx")
adls = glob("IMU Dataset_sample/sub*/ADLs/*xlsx")

df_falls = create_df(falls)
df_near_falls = create_df(near_falls)
df_adls = create_df(adls)

In [2]:
'''
1. Assumed ADLs and Near-Falls as Non-Falls.
2. Labelled Falls as 1 and Non-Falls as 0.
3. Combined ADLs and Near-Falls into a single dataframe named df_non_falls.
I am still not convinced with our defination of fall and non-fall, maybe we can
come up with a more sophisticated one.
'''
df_falls['label'] = 1
df_near_falls['label'] = 0
df_adls['label'] = 0
df_non_falls = df_near_falls.append(df_adls, ignore_index=True)

In [3]:
'''
Performed feature selection and dimensionality
reduction. For the time being I am sticking with Chi-Squared test 
and PCA as done in Assignment 5. I wanted to apply a voting mechanism by 
taking in consideration the top 10 features given by multiple statistical measures.
I don't know if this makes sense, but I don't have knowledge about all
the measures right now. Once I read more about it, maybe I will try to redo this part.
You guys can also look into this.
Before going further with feature selection, I will append df_falls with df_non_falls.
I will also shuffle the data.
'''
df_data = df_falls.append(df_non_falls, ignore_index=True)
df_data = df_data.sample(frac=1).reset_index(drop=True)

In [4]:
'''
Feature Selection
I just got to know chi squared test can't be applied on non negative values. We have to
come up with something new.
'''
pd.options.display.max_rows = 4000
df_data.isnull().sum()
# Lucky us there are no null values


Time                                  0
r.ankle Acceleration X (m/s^2)        0
r.ankle Acceleration Y (m/s^2)        0
r.ankle Acceleration Z (m/s^2)        0
r.ankle Angular Velocity X (rad/s)    0
r.ankle Angular Velocity Y (rad/s)    0
r.ankle Angular Velocity Z (rad/s)    0
r.ankle Magnetic Field X (uT)         0
r.ankle Magnetic Field Y (uT)         0
r.ankle Magnetic Field Z (uT)         0
l.ankle Acceleration X (m/s^2)        0
l.ankle Acceleration Y (m/s^2)        0
l.ankle Acceleration Z (m/s^2)        0
l.ankle Angular Velocity X (rad/s)    0
l.ankle Angular Velocity Y (rad/s)    0
l.ankle Angular Velocity Z (rad/s)    0
l.ankle Magnetic Field X (uT)         0
l.ankle Magnetic Field Y (uT)         0
l.ankle Magnetic Field Z (uT)         0
r.thigh Acceleration X (m/s^2)        0
r.thigh Acceleration Y (m/s^2)        0
r.thigh Acceleration Z (m/s^2)        0
r.thigh Angular Velocity X (rad/s)    0
r.thigh Angular Velocity Y (rad/s)    0
r.thigh Angular Velocity Z (rad/s)    0


In [5]:
'''
Resultant sqrt(X^2 + Y^2 + Z^2)
'''
import numpy as np
def resultant(df_data, x, y, z, string):
    df_data[string] = (df_data[x]**2 +\
                       df_data[y]**2 +\
                       df_data[z]**2).apply(np.sqrt)
        
#df_data['r.ankle Acceleration'] = (df_data['r.ankle Acceleration X (m/s^2)']**2 +\
                       #df_data['r.ankle Acceleration Y (m/s^2)']**2 +\
                       #df_data['r.ankle Acceleration Z (m/s^2)']**2).apply(np.sqrt)

resultant(df_data,'r.ankle Acceleration X (m/s^2)',
                   'r.ankle Acceleration Y (m/s^2)',
                   'r.ankle Acceleration Z (m/s^2)','r.ankle Acceleration (m/s^2)')
resultant(df_data,'r.ankle Angular Velocity X (rad/s)',
                   'r.ankle Angular Velocity Y (rad/s)',
                   'r.ankle Angular Velocity Z (rad/s)','r.ankle Angular Velocity (rad/s)')
resultant(df_data,'r.ankle Magnetic Field X (uT)',
                   'r.ankle Magnetic Field Y (uT)',
                   'r.ankle Magnetic Field Z (uT)','r.ankle Magnetic Field (uT)')
resultant(df_data,'l.ankle Acceleration X (m/s^2)',
                   'l.ankle Acceleration Y (m/s^2)',
                   'l.ankle Acceleration Z (m/s^2)','l.ankle Acceleration (m/s^2)')
resultant(df_data,'l.ankle Angular Velocity X (rad/s)',
                   'l.ankle Angular Velocity Y (rad/s)',
                   'l.ankle Angular Velocity Z (rad/s)','l.ankle Angular Velocity (rad/s)')
resultant(df_data,'l.ankle Magnetic Field X (uT)',
                   'l.ankle Magnetic Field Y (uT)',
                   'l.ankle Magnetic Field Z (uT)','l.ankle Magnetic Field (uT)')
resultant(df_data,'r.thigh Acceleration X (m/s^2)',
                   'r.thigh Acceleration Y (m/s^2)',
                   'r.thigh Acceleration Z (m/s^2)','r.thigh Acceleration (m/s^2)')
resultant(df_data,'r.thigh Angular Velocity X (rad/s)',
                   'r.thigh Angular Velocity Y (rad/s)',
                   'r.thigh Angular Velocity Z (rad/s)','r.thigh Angular Velocity (rad/s)')
resultant(df_data,'r.thigh Magnetic Field X (uT)',
                   'r.thigh Magnetic Field Y (uT)',
                   'r.thigh Magnetic Field Z (uT)','r.thigh Magnetic Field (uT)')
resultant(df_data,'l.thigh Acceleration X (m/s^2)',
                   'l.thigh Acceleration Y (m/s^2)',
                   'l.thigh Acceleration Z (m/s^2)','l.thigh Acceleration (m/s^2)')
resultant(df_data,'l.thigh Angular Velocity X (rad/s)',
                   'l.thigh Angular Velocity Y (rad/s)',
                   'l.thigh Angular Velocity Z (rad/s)','l.thigh Angular Velocity (rad/s)')
resultant(df_data,'l.thigh Magnetic Field X (uT)',
                   'l.thigh Magnetic Field Y (uT)',
                   'l.thigh Magnetic Field Z (uT)','l.thigh Magnetic Field (uT)')
resultant(df_data,'head Acceleration X (m/s^2)',
                   'head Acceleration Y (m/s^2)',
                   'head Acceleration Z (m/s^2)','head Acceleration (m/s^2)')
resultant(df_data,'head Angular Velocity X (rad/s)',
                   'head Angular Velocity Y (rad/s)',
                   'head Angular Velocity Z (rad/s)','head Angular Velocity (rad/s)')
resultant(df_data,'head Magnetic Field X (uT)',
                   'head Magnetic Field Y (uT)',
                   'head Magnetic Field Z (uT)','head Magnetic Field (uT)')
resultant(df_data,'sternum Acceleration X (m/s^2)',
                   'sternum Acceleration Y (m/s^2)',
                   'sternum Acceleration Z (m/s^2)','sternum Acceleration (m/s^2)')
resultant(df_data,'sternum Angular Velocity X (rad/s)',
                   'sternum Angular Velocity Y (rad/s)',
                   'sternum Angular Velocity Z (rad/s)','sternum Angular Velocity (rad/s)')
resultant(df_data,'sternum Magnetic Field X (uT)',
                   'sternum Magnetic Field Y (uT)',
                   'sternum Magnetic Field Z (uT)','sternum Magnetic Field (uT)')
resultant(df_data,'waist Acceleration X (m/s^2)',
                   'waist Acceleration Y (m/s^2)',
                   'waist Acceleration Z (m/s^2)','waist Acceleration (m/s^2)')
resultant(df_data,'waist Angular Velocity X (rad/s)',
                   'waist Angular Velocity Y (rad/s)',
                   'waist Angular Velocity Z (rad/s)','waist Angular Velocity (rad/s)')
resultant(df_data,'waist Magnetic Field X (uT)',
                   'waist Magnetic Field Y (uT)',
                   'waist Magnetic Field Z (uT)','waist Magnetic Field (uT)')

df_data.shape

(237567, 86)

In [35]:
df_data['label']

0         0
1         0
2         0
3         0
4         0
5         0
6         1
7         1
8         0
9         0
10        1
11        1
12        1
13        0
14        1
15        0
16        0
17        0
18        0
19        0
20        0
21        1
22        1
23        1
24        0
25        0
26        0
27        1
28        0
29        1
30        1
31        1
32        1
33        0
34        0
35        1
36        0
37        1
38        0
39        1
40        1
41        0
42        1
43        1
44        0
45        0
46        0
47        1
48        0
49        1
50        0
51        0
52        0
53        0
54        1
55        1
56        0
57        0
58        0
59        0
60        1
61        0
62        0
63        0
64        0
65        1
66        0
67        1
68        1
69        0
70        0
71        0
72        0
73        1
74        0
75        0
76        1
77        1
78        0
79        0
80        0
81        0
82        0
83  

In [6]:
label = df_data['label']
df_data.drop(labels=['label'], axis=1,inplace = True)
df_data.insert(85, 'label', label)

In [7]:
df_data.shape

(237567, 86)

In [39]:
X = df_data.drop(['label'], axis=1)
y = df_data.label
feature_name = X.columns.tolist()

In [40]:
'''
Pearson Correlation (hack not exactly correct)
https://stats.stackexchange.com/questions/113252/correlation-using-logistic-regression-and-pearson
No Normaliztion required
Missing values have to be imputed
'''

def cor_selector(X, y):
    cor_list = []
    # calculate the correlation with y for each feature
    for i in X.columns.tolist():
        cor = np.corrcoef(X[i], y)[0, 1]
        cor_list.append(cor)
    # feature name
    cor_feature = X.iloc[:,np.argsort(np.abs(cor_list))[-5:]].columns.tolist()
    # feature selection? 0 for not select, 1 for select
    cor_support = [True if i in cor_feature else False for i in feature_name]
    return cor_support, cor_feature

cor_support, cor_feature = cor_selector(X, y)
print(str(len(cor_feature)), 'selected features')
print(cor_feature)


5 selected features
['r.ankle Acceleration X (m/s^2)', 'r.thigh Acceleration X (m/s^2)', 'waist Acceleration X (m/s^2)', 'sternum Acceleration X (m/s^2)', 'head Acceleration X (m/s^2)']


In [41]:
'''
Wrapper 
RFE
'''
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

X_norm = MinMaxScaler().fit_transform(X)
rfe_selector = RFE(estimator=LogisticRegression(), n_features_to_select=5, step=10, verbose=5)
rfe_selector.fit(X_norm, y)

rfe_support = rfe_selector.get_support()
rfe_feature = X.loc[:,rfe_support].columns.tolist()
print(str(len(rfe_feature)), 'selected features')

/home/salil/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


Fitting estimator with 85 features.


/home/salil/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Fitting estimator with 75 features.


/home/salil/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


...

Fitting estimator with 25 features.


/home/salil/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Fitting estimator with 15 features.


/home/salil/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/salil/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


5 selected features


In [ ]:
rfe_feature

In [38]:
'''
Random Forest
'''

from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

embeded_rf_selector = SelectFromModel(RandomForestClassifier(n_estimators=5), threshold='1.25*median')
embeded_rf_selector.fit(X, y)

embeded_rf_support = embeded_rf_selector.get_support()
embeded_rf_feature = X.loc[:,embeded_rf_support].columns.tolist()
print(str(len(embeded_rf_feature)), 'selected features')

37 selected features


In [39]:
embeded_rf_feature

['Time',
 'r.ankle Acceleration X (m/s^2)',
 'r.ankle Acceleration Y (m/s^2)',
 'r.ankle Magnetic Field X (uT)',
 'r.ankle Magnetic Field Y (uT)',
 'r.ankle Magnetic Field Z (uT)',
 'l.ankle Acceleration Z (m/s^2)',
 'l.ankle Magnetic Field X (uT)',
 'l.ankle Magnetic Field Y (uT)',
 'l.ankle Magnetic Field Z (uT)',
 'r.thigh Acceleration X (m/s^2)',
 'r.thigh Acceleration Y (m/s^2)',
 'r.thigh Magnetic Field X (uT)',
 'r.thigh Magnetic Field Y (uT)',
 'r.thigh Magnetic Field Z (uT)',
 'l.thigh Acceleration X (m/s^2)',
 'l.thigh Acceleration Y (m/s^2)',
 'l.thigh Magnetic Field X (uT)',
 'l.thigh Magnetic Field Y (uT)',
 'l.thigh Magnetic Field Z (uT)',
 'head Acceleration Y (m/s^2)',
 'head Acceleration Z (m/s^2)',
 'head Magnetic Field Y (uT)',
 'head Magnetic Field Z (uT)',
 'sternum Acceleration X (m/s^2)',
 'sternum Acceleration Y (m/s^2)',
 'sternum Acceleration Z (m/s^2)',
 'waist Acceleration Y (m/s^2)',
 'waist Acceleration Z (m/s^2)',
 'waist Magnetic Field X (uT)',
 'waist M

In [ ]:
'''
LightGBM
'''

from sklearn.feature_selection import SelectFromModel
from lightgbm import LGBMClassifier

lgbc=LGBMClassifier(n_estimators=500, learning_rate=0.05, num_leaves=32, colsample_bytree=0.2,
            reg_alpha=3, reg_lambda=1, min_split_gain=0.01, min_child_weight=40)

embeded_lgb_selector = SelectFromModel(lgbc, threshold='1.25*median')
embeded_lgb_selector.fit(X, y)

embeded_lgb_support = embeded_lgb_selector.get_support()
embeded_lgb_feature = X.loc[:,embeded_lgb_support].columns.tolist()
print(str(len(embeded_lgb_feature)), 'selected features')

In [40]:
'''
Summary
'''
#feature_selection_df = pd.DataFrame({'Feature':feature_name, 'Pearson':cor_support, 'RFE':rfe_support, 'Logistics':embeded_lr_support,
                                    #'Random Forest':embeded_rf_support, 'LightGBM':embeded_lgb_support})
# count the selected times for each feature
feature_selection_df = pd.DataFrame({'Feature':feature_name, 'Pearson':cor_support, 'RFE':rfe_support,
                                    'Random Forest':embeded_rf_support, 'LightGBM':embeded_lgb_support})
feature_selection_df['Total'] = np.sum(feature_selection_df, axis=1)
# display the top 100
feature_selection_df = feature_selection_df.sort_values(['Total','Feature'] , ascending=False)
feature_selection_df.index = range(1, len(feature_selection_df)+1)
feature_selection_df.head(5)

Feature  Pearson    RFE  Random Forest  Total
1  sternum Acceleration Z (m/s^2)    False   True           True      2
2  sternum Acceleration X (m/s^2)     True  False           True      2
3  r.thigh Acceleration X (m/s^2)     True  False           True      2
4  r.ankle Acceleration X (m/s^2)     True  False           True      2
5     head Acceleration Z (m/s^2)    False   True           True      2

In [10]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df_data, test_size=0.2)
X_train = train.drop(['label'], axis=1)
y_train = train.label
X_test = test.drop(['label'], axis=1)
y_test = test.label

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

LogR = LogisticRegression(penalty='l2', C= 1.0, max_iter=1, n_jobs=1, tol=0.001)
LogR = LogR.fit(X_train, y_train)
y_pred = LogR.predict(X_test)
print(classification_report(y_test, y_pred, digits=3))
print(LogR.score(X_test, y_test))

/home/salil/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/salil/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/salil/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0      0.661     1.000     0.796     31418
           1      0.000     0.000     0.000     16096

   micro avg      0.661     0.661     0.661     47514
   macro avg      0.331     0.500     0.398     47514
weighted avg      0.437     0.661     0.526     47514

0.6612366881340237


In [ ]:
from sklearn import svm
clf = svm.SVC(gamma='scale')
clf.fit(X_train, y_train) 
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, digits=3))
print(clf.score(X_test, y_test))